# MCF LP Problem Formulation

## Basic definitions

**Definition**: MCF Net

We define a **net** to be a tuple $(V, E, c, P, S, T, \sigma, d)$ where
    
 - $V$ is a non-empty set of vertices.
 - $G \subseteq V \times V - \{ (v, v) \mid v \in V \}$ is a set of (oriented) edges (self-loops are not included).
 - $c$ is a function $c : E \to \mathbb{R}_{0}^{+}$ which assigns a *capacity* to each edge.
 - $P$ is a non-empty set of *products*.
 - $S, T \subseteq V$ are in order a set of *source* and *destination* nodes
 - $\sigma$ is a function $\sigma : S \times P \to \mathbb{R}^{+}$ which assigns each supply node the amount of the given product available.
 - $d$ is a function $\delta : T \times P \to \mathbb{R}^{+}$ which assigns each demand node a demand for the given product.
 
 For simplicity we denote $\sigma_s^k := \sigma(s, k)$ and $d_t^k := d(s, k)$.
 
**Definition**: Neighbourhood

For oriented graph $G = (V, E)$ we define for each $v \in V$
$$
\begin{gather*}
N^{+}(v) = \{ u \mid (u, v) \in E \} \\
N^{-}(v) = \{ u \mid (v, u) \in E \}
\end{gather*}
$$
meaning $N^{+}$ is a set of all nodes with an edge comming into $v$ and $N^{-}$ is a set of all nodes for which $v$ has an outcomming edge.

**Definition**: Flow

Let $N = (V, E, c, P, S, T, \sigma, d)$ be a net. Then we define **flow** as a function
$$f : E \times P \to \mathbb{R}^{+}$$
satisfying the following conditions:
 - *the flow does not exceed edge cappacities*
     $$\forall e \in E : \sum\limits_{i \in P} f(e, i) \le c(e)$$
 - *the flow satisfies Kirchhoff's law for each procut and for all but source and destination nodes*
     $$\forall i \in P : \forall v \in (V - (S \cup T)) : \sum\limits_{u \in N^{+}(v)} f(uv, i) - \sum\limits_{u \in N^{-}(v)} f(vu, i) = 0$$
     
For simplicity we denote $f_e^k := f(e, k)$.

## LP Formulation

We are interested in a problem where demands of the destination nodes might not be met. We allow this and measure it with slack variables $\xi$. We will want to minimize these values and transport as much of the products as we can.

In the LP formulation we will have the following **variables**:

 - $f_e^k \in \mathbb{Q}^{+}, f_e^k \ge 0$ $\sim$ representing the flow of the product $k$ through the edge $e$
 - $\xi_v^k \in \mathbb{Q}^{+}, \xi_v^k \ge 0$ $\sim$ the slack variable for node $v \in T$ and product $k \in P$ (how much of the product are we still missing)
    
**Objective function**
$$ \min \sum_{(v, k) \in T \times P} \xi_v^k$$

**Given the conditions**
    
 - *respecting edge capacities*
 $$\forall e \in E : \sum\limits_{k \in P} f_e^k \le c(e)$$
 - *Kirchhoff's law*
 $$\forall v \in (V - (S \cup T)) : \forall k \in P : \sum\limits_{u \in N^{+}(v)} f_{uv}^k - \sum\limits_{u \in N^{-}(v)} f_{vu}^k = 0$$
 - *demands are satisfied given some slack*
 $$\forall t \in T : \forall k \in P : \sum_{u \in N^{+}(t)} f_{ut}^k + \xi_t^k = d_t^k$$
 - *supplies are not exhausted*
  $$\forall s \in S : \forall k \in P : \sum_{u \in N^{-}(s)} f_{su}^k \le \sigma_t^k$$
  
Adding additional variables $\varepsilon_e \ge 0$ for every edge in capacity conditions and $\varphi_s^k$ for every $s \in S$ and $k \in P$.

We arrive at the final LP formulation

**Objective function**
$$ \max \sum_{(v, k) \in T \times P} -\xi_v^k$$

**Conditions**
$$
\begin{align*}
\forall e \in E : &\sum\limits_{k \in P} f_e^k + \varepsilon_e = c(e) \\
\forall v \in (V - (S \cup T)) : \forall k \in P : &\sum\limits_{u \in N^{+}(v)} f_{uv}^k - \sum\limits_{u \in N^{-}(v)} f_{vu}^k = 0 \\
\forall t \in T : \forall k \in P : &\sum_{u \in N^{+}(t)} f_{ut}^k + \xi_t^k = d_t^k \\
\forall s \in S : \forall k \in P : &\sum_{u \in N^{-}(s)} f_{su}^k + \varphi_{s}^k = \sigma_t^k
\end{align*}
$$

**Variables**
$$
\begin{align*}
\forall e \in E : \forall k \in P &: f_e^k \in \mathbb{Q}^{+} && f_e^k \ge 0 \\
\forall t \in T : \forall k \in P &: \xi_t^k \in \mathbb{Q}^{+} && \xi_k \ge 0 \\
\forall e \in E &: \varepsilon_e \in \mathbb{Q}^{+} && \varepsilon_e \ge 0 \\
\forall s \in S : \forall k \in P &: \varphi_{s}^k \in \mathbb{Q}^{+} && \varphi_{s}^k \ge 0
\end{align*}
$$

Denoting $n$ the number of nodes and $m$ the number of edges, we have together $m + |P|n$ conditions and $(|P| + 1)m + (|S| + |D|)|P|$ variables.

### Collecting relevant information from instances

In [20]:
import numpy as np
import sympy as sp
from pathlib import Path
from mcf_simplex_analyzer.load_instance import load_instance
from collections import defaultdict
from fractions import Fraction

from pprint import pprint

# Path to data directory
instances_path = Path("example/")

assert instances_path.exists()

instance_format = "mnetgen"
nod_file = instances_path / (instance_format + ".nod")
arc_file = instances_path / (instance_format + ".arc")
sup_file = instances_path / (instance_format + ".sup")
mut_file = instances_path / (instance_format + ".mut")

instance = load_instance(instance_format, nod_file, arc_file, sup_file, mut_file)
print(instance.info)

InstanceInfo(products_no=4, nodes_no=64, links_no=196, bundled_links_no=84)


In [25]:
# Collect edge capacities
capacities = {}

for arc in instance.arcs:
    fromnode, tonode, commodity, cost, individual_capacity, mutual_ptr = arc
    if fromnode == -1 or tonode == -1 or commodity == -1:
        print(arc)
        
    mutual, capacity = capacities.get((fromnode, tonode), (-1, -1))
    mutual = mutual + instance.mutual.capacity[mutual_ptr - 1] if mutual_ptr != 0 else -1
    capacity = capacity + individual_capacity if capacity != -1 else individual_capacity
    
    capacities[(fromnode, tonode)] = (mutual, capacity)

for key in capacities:
    mutual, total = capacities[key]
    capacities[key] = max(mutual, total) if mutual != -1 else total
    

print(capacities)
print(len(capacities))

{(1, 40): Fraction(64, 1), (40, 35): Fraction(511, 1), (35, 43): Fraction(511, 1), (2, 36): Fraction(647, 1), (36, 45): Fraction(-1, 1), (45, 56): Fraction(647, 1), (45, 33): Fraction(158, 1), (45, 35): Fraction(1159, 1), (3, 22): Fraction(511, 1), (22, 39): Fraction(511, 1), (4, 40): Fraction(511, 1), (40, 41): Fraction(511, 1), (41, 23): Fraction(62, 1), (23, 36): Fraction(31, 1), (23, 39): Fraction(-1, 1), (23, 63): Fraction(511, 1), (5, 29): Fraction(30, 1), (29, 64): Fraction(511, 1), (6, 33): Fraction(511, 1), (33, 51): Fraction(30, 1), (7, 38): Fraction(95, 1), (38, 17): Fraction(31, 1), (17, 46): Fraction(511, 1), (17, 62): Fraction(64, 1), (8, 25): Fraction(511, 1), (25, 36): Fraction(-1, 1), (36, 20): Fraction(511, 1), (20, 55): Fraction(29, 1), (20, 60): Fraction(63, 1), (20, 57): Fraction(62, 1), (9, 38): Fraction(31, 1), (38, 32): Fraction(511, 1), (32, 40): Fraction(511, 1), (32, 49): Fraction(-1, 1), (10, 40): Fraction(29, 1), (40, 28): Fraction(575, 1), (28, 50): Fracti

In [23]:
# Collect source and destination vertices

source = defaultdict(dict)
destination = defaultdict(dict)

for supply in instance.supply:
    s, t, k, f = supply
    if s == -1:
        destination[t][k] = f
        
    if t == -1:
        source[s][k] = f
        
print(source)
print(destination)

defaultdict(<class 'dict'>, {1: {1: Fraction(32, 1), 2: Fraction(32, 1), 3: Fraction(32, 1), 4: Fraction(32, 1)}, 2: {1: Fraction(32, 1), 2: Fraction(32, 1), 3: Fraction(32, 1), 4: Fraction(66, 1)}, 3: {1: Fraction(32, 1), 2: Fraction(32, 1), 3: Fraction(32, 1), 4: Fraction(32, 1)}, 4: {1: Fraction(32, 1), 2: Fraction(32, 1), 3: Fraction(32, 1), 4: Fraction(32, 1)}, 5: {1: Fraction(32, 1), 2: Fraction(32, 1), 3: Fraction(32, 1), 4: Fraction(32, 1)}, 6: {1: Fraction(32, 1), 2: Fraction(32, 1), 3: Fraction(32, 1), 4: Fraction(32, 1)}, 7: {1: Fraction(32, 1), 2: Fraction(32, 1), 3: Fraction(32, 1), 4: Fraction(32, 1)}, 8: {1: Fraction(32, 1), 2: Fraction(32, 1), 3: Fraction(32, 1), 4: Fraction(32, 1)}, 9: {1: Fraction(32, 1), 2: Fraction(32, 1), 3: Fraction(32, 1), 4: Fraction(32, 1)}, 10: {1: Fraction(32, 1), 2: Fraction(32, 1), 3: Fraction(32, 1), 4: Fraction(48, 1)}, 11: {1: Fraction(32, 1), 2: Fraction(32, 1), 3: Fraction(32, 1), 4: Fraction(32, 1)}, 12: {1: Fraction(32, 1), 2: Fracti

In [27]:
in_neighbours = defaultdict(set)
out_neighbours = defaultdict(set)

for arc in instance.arcs:
    fromnode, tonode, _, _, _, _ = arc
    in_neighbours[tonode].add(fromnode)
    out_neighbours[fromnode].add(tonode)

print(in_neighbours)
print(out_neighbours)

defaultdict(<class 'set'>, {40: {32, 1, 33, 35, 4, 10, 47, 15, 49, 21}, 35: {32, 33, 40, 45, 18, 19, 25}, 43: {33, 34, 35, 45, 24}, 36: {33, 2, 44, 23, 25, 26}, 45: {33, 36, 49, 20, 30}, 56: {33, 45}, 33: {6, 42, 44, 45, 48, 18, 31}, 22: {49, 3}, 39: {33, 34, 41, 22, 23}, 41: {33, 40, 46, 14, 19, 27}, 23: {41}, 63: {33, 44, 54, 23}, 29: {5, 46}, 64: {33, 51, 27, 29, 30}, 51: {33, 21, 38}, 38: {33, 7, 9, 15, 19}, 17: {54, 11, 38}, 46: {33, 45, 17, 21, 27, 28, 29}, 62: {17, 21, 30, 33}, 25: {8, 4, 21}, 20: {36}, 55: {33, 43, 47, 20, 60, 31}, 60: {33, 7, 9, 47, 20}, 57: {33, 35, 44, 20, 26}, 32: {26, 3, 38}, 49: {32, 33, 45}, 28: {40, 26}, 50: {33, 34, 28}, 37: {33, 26, 35, 28}, 18: {11}, 53: {48, 33, 35}, 52: {19, 33, 35}, 44: {33, 35, 12, 13, 47, 17, 24}, 34: {33, 8, 44, 22, 30, 31}, 48: {33, 34, 46, 30}, 58: {24, 33, 44, 47}, 19: {46, 58, 14}, 47: {32, 33, 34, 39, 16, 24, 30}, 27: {41, 53}, 54: {3, 33, 35, 45}, 24: {21}, 30: {23}, 26: {25, 28, 38}, 31: {56, 50, 37}, 21: {33}, 42: {33, 